<a href="https://colab.research.google.com/github/MennaEwas/GoogleMLAcademy/blob/main/huggingFaceFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[dev]

In [ ]:
pip install --pre torchaudio -f https://download.pytorch.org/whl/nightly/torch_nightly.html

In [ ]:
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification,Trainer,TrainingArguments
checkpoint='bert-base-uncased'

In [ ]:
model=AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)
tokenzier=AutoTokenizer.from_pretrained(checkpoint)

In [ ]:


raw_datasets = load_dataset("glue", "mrpc")

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-d50264bc56a2829e.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-049a95e30376791f.arrow


In [ ]:
training_args=TrainingArguments('test-trainer',evaluation_strategy="epoch",
                                per_device_train_batch_size=16,
                                per_device_eval_batch_size=16,
                                learning_rate=2e-5,
                                weight_decay=0.01,
                                num_train_epochs=5)
trainer= Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
  
)

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [ ]:
import numpy as np
preds=np.argmax(predictions.predictions,axis=-1)

In [ ]:
metric = load_metric("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.6838235294117647, 'f1': 0.8122270742358079}

In [ ]:
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer.train()